In [1]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\box_square_functions.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie, LinearAlgebra, LinearSolve

pathtofile = "C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab\\"
I0_hom = load(pathtofile * "I0_hom_hom.jld")["data"]
u0_hom = load(pathtofile * "U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [16]:
index_point_from_curve = 1

const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305; const U0 = u0_hom[index_point_from_curve]

t = 2000.0; tspan = [0.0, t]
u0 = [0.0, 0.0, 0.0]
ϵ_box = 1.0e-5; ϵ_shift = 1.0e-2;

In [18]:
number_points_on_side = 10
total_count_points = number_points_on_side * 2

I0start = I0_hom[index_point_from_curve]
I0end = I0_hom[index_point_from_curve] - ϵ_shift
len_range = 5

p_range =  range(I0start, I0end, length = len_range)

size_vector = len_range * total_count_points;

In [19]:
dots_u0 = zeros(size_vector, 3)
check_events = Vector{Bool}(undef, size_vector)
time_events = zeros(size_vector)
dots_on_event = zeros(size_vector, 3)
αs = zeros(size_vector, 3)
norms = zeros(size_vector, 3)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile  -  20:04:28", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip

In [ ]:
for p_index in range(1, length(p_range), step = 1)

    if p_index == 1
        global start_internal_index = 1
    end

    println("index param: $(p_index)"); flush(stdout)

    p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, p_range[p_index]]

    fixedpoint = get_fixed_point(TM, jacob_TM_, p, u0)
    A = get_matrix(fixedpoint[1], p, jacob_TM_, 2)

    dots_right_side, array_α_vec_right,
    dots_left_side, array_α_vec_left,
    dots_up_side, array_α_vec_up,
    dots_down_side, array_α_vec_down = get_arrays_dots(number_points_on_side, 3)

    fill_side_square(ϵ_box, fixedpoint[1], number_points_on_side, A, 
    dots_right_side, array_α_vec_right, dots_left_side, array_α_vec_left,
    dots_up_side, array_α_vec_up, dots_down_side, array_α_vec_down)


    condition = make_event(fixedpoint[1], ϵ_box, A)
    cb = ContinuousCallback(condition, nothing, affect!)

    up_and_down_side = cat(dots_up_side, dots_down_side, dims = 1)
    final_internal_index = total_count_points * p_index

    println("start internal index: $(start_internal_index)"); flush(stdout)
    println("final internal index: $(final_internal_index)"); flush(stdout)

    start_internal_index = trajectory_from_sides(p, A, up_and_down_side, final_internal_index,
                    fixedpoint[1], cb, start_internal_index)
                    
    println("-------------------------------"); flush(stdout)

end

In [5]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 24; number of no event: 76


In [6]:
filename = "index_point_from_curve $(index_point_from_curve)"
filename = filename*".jld2"

"index_point_from_curve 1.jld2"

In [7]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\manifolds\\TM\\box square\\")
jldsave(filename; dots_u0, check_events, time_events, dots_on_event ,αs, norms)